In [2]:

from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('WeatherDataProcessing').getOrCreate()

In [4]:
df = spark.read.csv(r'C:\Users\aravi\Desktop\Assignment\GlobalWeatherRepository.csv', header=True, inferSchema=True)

In [5]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: string (nullable = true)
 |-- last_updated_epoch: integer (nullable = true)
 |-- last_updated: timestamp (nullable = true)
 |-- temperature_celsius: double (nullable = true)
 |-- temperature_fahrenheit: double (nullable = true)
 |-- condition_text: string (nullable = true)
 |-- wind_mph: double (nullable = true)
 |-- wind_kph: double (nullable = true)
 |-- wind_degree: integer (nullable = true)
 |-- wind_direction: string (nullable = true)
 |-- pressure_mb: double (nullable = true)
 |-- pressure_in: double (nullable = true)
 |-- precip_mm: double (nullable = true)
 |-- precip_in: double (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- cloud: integer (nullable = true)
 |-- feels_like_celsius: double (nullable = true)
 |-- feels_like_fahrenheit: double (nullable = true)
 |-- visibility_km: d

In [6]:
from pyspark.sql.functions import to_date, col

In [7]:
df = df.withColumn('date_only', to_date(col('last_updated')))

In [8]:
df.show()

+-------------------+----------------+--------+---------+--------------------+------------------+-------------------+-------------------+----------------------+--------------------+--------+--------+-----------+--------------+-----------+-----------+---------+---------+--------+-----+------------------+---------------------+-------------+----------------+--------+--------+--------+---------------------------+-----------------+----------------------------+---------------------------+-----------------+----------------+------------------------+--------------------------+--------+--------+--------+----------+--------------+-----------------+----------+
|            country|   location_name|latitude|longitude|            timezone|last_updated_epoch|       last_updated|temperature_celsius|temperature_fahrenheit|      condition_text|wind_mph|wind_kph|wind_degree|wind_direction|pressure_mb|pressure_in|precip_mm|precip_in|humidity|cloud|feels_like_celsius|feels_like_fahrenheit|visibility_km|vis

In [9]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: string (nullable = true)
 |-- last_updated_epoch: integer (nullable = true)
 |-- last_updated: timestamp (nullable = true)
 |-- temperature_celsius: double (nullable = true)
 |-- temperature_fahrenheit: double (nullable = true)
 |-- condition_text: string (nullable = true)
 |-- wind_mph: double (nullable = true)
 |-- wind_kph: double (nullable = true)
 |-- wind_degree: integer (nullable = true)
 |-- wind_direction: string (nullable = true)
 |-- pressure_mb: double (nullable = true)
 |-- pressure_in: double (nullable = true)
 |-- precip_mm: double (nullable = true)
 |-- precip_in: double (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- cloud: integer (nullable = true)
 |-- feels_like_celsius: double (nullable = true)
 |-- feels_like_fahrenheit: double (nullable = true)
 |-- visibility_km: d

In [10]:
df = df.withColumn("date", df["date_only"].cast("date"))

In [11]:
from pyspark.sql.functions import year
df = df.withColumn("year", year(df["date_only"]))

In [12]:
df.show()

+-------------------+----------------+--------+---------+--------------------+------------------+-------------------+-------------------+----------------------+--------------------+--------+--------+-----------+--------------+-----------+-----------+---------+---------+--------+-----+------------------+---------------------+-------------+----------------+--------+--------+--------+---------------------------+-----------------+----------------------------+---------------------------+-----------------+----------------+------------------------+--------------------------+--------+--------+--------+----------+--------------+-----------------+----------+----------+----+
|            country|   location_name|latitude|longitude|            timezone|last_updated_epoch|       last_updated|temperature_celsius|temperature_fahrenheit|      condition_text|wind_mph|wind_kph|wind_degree|wind_direction|pressure_mb|pressure_in|precip_mm|precip_in|humidity|cloud|feels_like_celsius|feels_like_fahrenheit|v

In [13]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: string (nullable = true)
 |-- last_updated_epoch: integer (nullable = true)
 |-- last_updated: timestamp (nullable = true)
 |-- temperature_celsius: double (nullable = true)
 |-- temperature_fahrenheit: double (nullable = true)
 |-- condition_text: string (nullable = true)
 |-- wind_mph: double (nullable = true)
 |-- wind_kph: double (nullable = true)
 |-- wind_degree: integer (nullable = true)
 |-- wind_direction: string (nullable = true)
 |-- pressure_mb: double (nullable = true)
 |-- pressure_in: double (nullable = true)
 |-- precip_mm: double (nullable = true)
 |-- precip_in: double (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- cloud: integer (nullable = true)
 |-- feels_like_celsius: double (nullable = true)
 |-- feels_like_fahrenheit: double (nullable = true)
 |-- visibility_km: d

In [14]:
df.select("year").show()

+----+
|year|
+----+
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
|2024|
+----+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import month
df = df.withColumn("month", month(df["date"]))

In [16]:
df.select("month").show()

+-----+
|month|
+-----+
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
|    5|
+-----+
only showing top 20 rows



In [17]:
from pyspark.sql.functions import dayofmonth
df = df.withColumn("day", dayofmonth(df["date"]))

In [18]:
df.select("date", "day").show()

+----------+---+
|      date|day|
+----------+---+
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
|2024-05-16| 16|
+----------+---+
only showing top 20 rows



In [19]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: string (nullable = true)
 |-- last_updated_epoch: integer (nullable = true)
 |-- last_updated: timestamp (nullable = true)
 |-- temperature_celsius: double (nullable = true)
 |-- temperature_fahrenheit: double (nullable = true)
 |-- condition_text: string (nullable = true)
 |-- wind_mph: double (nullable = true)
 |-- wind_kph: double (nullable = true)
 |-- wind_degree: integer (nullable = true)
 |-- wind_direction: string (nullable = true)
 |-- pressure_mb: double (nullable = true)
 |-- pressure_in: double (nullable = true)
 |-- precip_mm: double (nullable = true)
 |-- precip_in: double (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- cloud: integer (nullable = true)
 |-- feels_like_celsius: double (nullable = true)
 |-- feels_like_fahrenheit: double (nullable = true)
 |-- visibility_km: d

In [20]:
df.select("precip_mm").show()

+---------+
|precip_mm|
+---------+
|      0.0|
|      0.1|
|      0.0|
|      0.3|
|      0.0|
|     0.02|
|      0.0|
|     0.13|
|      0.0|
|      0.0|
|      0.0|
|      0.0|
|      0.0|
|      0.0|
|      0.0|
|      0.0|
|     0.25|
|      0.0|
|     0.02|
|     0.16|
+---------+
only showing top 20 rows



In [21]:
df.select("temperature_fahrenheit").show()

+----------------------+
|temperature_fahrenheit|
+----------------------+
|                  79.8|
|                  66.2|
|                  73.4|
|                  43.3|
|                  78.8|
|                  78.8|
|                  46.4|
|                  66.2|
|                  48.2|
|                  60.8|
|                  62.6|
|                  80.6|
|                  96.8|
|                 101.2|
|                  82.4|
|                  62.6|
|                  59.0|
|                  78.9|
|                  82.4|
|                  64.8|
+----------------------+
only showing top 20 rows



In [22]:
from pyspark.sql.types import FloatType
df = df.withColumn("precip_in", df["precip_in"].cast(FloatType()))

In [23]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: string (nullable = true)
 |-- last_updated_epoch: integer (nullable = true)
 |-- last_updated: timestamp (nullable = true)
 |-- temperature_celsius: double (nullable = true)
 |-- temperature_fahrenheit: double (nullable = true)
 |-- condition_text: string (nullable = true)
 |-- wind_mph: double (nullable = true)
 |-- wind_kph: double (nullable = true)
 |-- wind_degree: integer (nullable = true)
 |-- wind_direction: string (nullable = true)
 |-- pressure_mb: double (nullable = true)
 |-- pressure_in: double (nullable = true)
 |-- precip_mm: double (nullable = true)
 |-- precip_in: float (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- cloud: integer (nullable = true)
 |-- feels_like_celsius: double (nullable = true)
 |-- feels_like_fahrenheit: double (nullable = true)
 |-- visibility_km: do

In [24]:
df = df.na.fill({"precip_in": 0.0})

In [25]:
df.select("precip_in").show(10)

+---------+
|precip_in|
+---------+
|      0.0|
|      0.0|
|      0.0|
|     0.01|
|      0.0|
|      0.0|
|      0.0|
|     0.01|
|      0.0|
|      0.0|
+---------+
only showing top 10 rows



In [26]:
from pyspark.sql.functions import mean, sum

# Aggregate by month
monthly_data = df.groupBy("year", "month").agg(
    mean("temperature_fahrenheit").alias("avg_temperature"),
    sum("precip_in").alias("total_precipitation")
)

# Select and show the average temperature
#monthly_data.select("avg_temperature").show()


In [27]:
monthly_data.show()

+----+-----+------------------+-------------------+
|year|month|   avg_temperature|total_precipitation|
+----+-----+------------------+-------------------+
|2024|    7| 80.24264468733195|  34.74999968148768|
|2024|    5| 77.27550030321397| 20.219999799504876|
|2024|    6| 79.62250731875338| 34.859999703243375|
|2024|    8|  80.2230934656741|  38.51999959908426|
|2024|    9| 77.22641734972692|  37.29999973252416|
|2024|   10| 72.20408637873754| 28.579999689012766|
|2024|   12| 63.95377998345755|  25.28999970294535|
|2024|   11|  66.9664947567473| 30.089999679476023|
|2025|    1|63.309157509157394| 28.509999683126807|
|2025|    2| 63.07938441808273| 11.769999854266644|
+----+-----+------------------+-------------------+



In [28]:
ordered_monthly_data = monthly_data.orderBy("month")

In [29]:
ordered_monthly_data.show()

+----+-----+------------------+-------------------+
|year|month|   avg_temperature|total_precipitation|
+----+-----+------------------+-------------------+
|2025|    1|63.309157509157394| 28.509999683126807|
|2025|    2| 63.07938441808273| 11.769999854266644|
|2024|    5| 77.27550030321397| 20.219999799504876|
|2024|    6| 79.62250731875338| 34.859999703243375|
|2024|    7| 80.24264468733195|  34.74999968148768|
|2024|    8|  80.2230934656741|  38.51999959908426|
|2024|    9| 77.22641734972692|  37.29999973252416|
|2024|   10| 72.20408637873754| 28.579999689012766|
|2024|   11|  66.9664947567473| 30.089999679476023|
|2024|   12| 63.95377998345755|  25.28999970294535|
+----+-----+------------------+-------------------+



In [30]:

daily_precipitation = df.groupBy("year", "month", "day").agg(
    sum("precip_in").alias("total_precipitation")
)


In [31]:

# Group by year and month and calculate the average temperature and total precipitation
monthly_data = df.groupBy("year", "month").agg(
    mean("temperature_fahrenheit").alias("avg_temperature"),
    sum("precip_in").alias("total_precipitation")
)

# Show the result
monthly_data.show()



+----+-----+------------------+-------------------+
|year|month|   avg_temperature|total_precipitation|
+----+-----+------------------+-------------------+
|2024|    7| 80.24264468733195|  34.74999968148768|
|2024|    5| 77.27550030321397| 20.219999799504876|
|2024|    6| 79.62250731875338| 34.859999703243375|
|2024|    8|  80.2230934656741|  38.51999959908426|
|2024|    9| 77.22641734972692|  37.29999973252416|
|2024|   10| 72.20408637873754| 28.579999689012766|
|2024|   12| 63.95377998345755|  25.28999970294535|
|2024|   11|  66.9664947567473| 30.089999679476023|
|2025|    1|63.309157509157394| 28.509999683126807|
|2025|    2| 63.07938441808273| 11.769999854266644|
+----+-----+------------------+-------------------+



In [32]:
# Daily Aggregation
daily_data = df.groupBy("year", "month", "day").agg(
    mean("temperature_fahrenheit").alias("avg_temperature"),
    sum("precip_in").alias("total_precipitation")
)
daily_data.show()

+----+-----+---+-----------------+-------------------+
|year|month|day|  avg_temperature|total_precipitation|
+----+-----+---+-----------------+-------------------+
|2024|    6|  8|             79.8| 1.3899999968707561|
|2024|    5| 24|77.16051282051282|  1.359999991953373|
|2024|    5| 21|77.69076923076922|  1.099999975413084|
|2024|    6|  3|79.02615384615382| 1.4200000055134296|
|2024|    6| 30|80.44895833333331| 1.1399999912828207|
|2024|    6| 26|80.67305699481864| 0.7699999865144491|
|2024|    7|  5|79.20454545454552|  1.019999984651804|
|2024|    6| 27|80.94923076923077|  1.529999990016222|
|2024|    7|  1|79.65384615384616| 1.2999999821186066|
|2024|    6|  6|79.76994818652852| 0.4599999915808439|
|2024|    6| 28|80.58615384615386| 0.9899999853223562|
|2024|    8|  7|80.10512820512828| 0.8099999874830246|
|2024|    5| 18|77.63658536585362| 0.9999999813735485|
|2024|    5| 23|77.12000000000003| 1.2799999918788671|
|2024|    6| 13|79.70717948717949|  1.080000001937151|
|2024|    

In [33]:
# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Display the Pandas DataFrame
pandas_df


,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,sunset,moonrise,moonset,moon_phase,moon_illumination,date_only,date,year,month,day
0,Afghanistan,Kabul,34.5200,69.1800,Asia/Kabul,1715849100,2024-05-16 13:15:00,26.6,79.8,Partly Cloudy,...,06:50 PM,12:12 PM,01:11 AM,Waxing Gibbous,55,2024-05-16,2024-05-16,2024,5,16
1,Albania,Tirana,41.3300,19.8200,Europe/Tirane,1715849100,2024-05-16 10:45:00,19.0,66.2,Partly cloudy,...,07:54 PM,12:58 PM,02:14 AM,Waxing Gibbous,55,2024-05-16,2024-05-16,2024,5,16
2,Algeria,Algiers,36.7600,3.0500,Africa/Algiers,1715849100,2024-05-16 09:45:00,23.0,73.4,Sunny,...,07:50 PM,01:15 PM,02:14 AM,Waxing Gibbous,55,2024-05-16,2024-05-16,2024,5,16
3,Andorra,Andorra La Vella,42.5000,1.5200,Europe/Andorra,1715849100,2024-05-16 10:45:00,6.3,43.3,Light drizzle,...,09:11 PM,02:12 PM,03:31 AM,Waxing Gibbous,55,2024-05-16,2024-05-16,2024,5,16
4,Angola,Luanda,-8.8400,13.2300,Africa/Luanda,1715849100,2024-05-16 09:45:00,26.0,78.8,Partly cloudy,...,05:55 PM,01:17 PM,12:38 AM,Waxing Gibbous,55,2024-05-16,2024-05-16,2024,5,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53589,Venezuela,Caracas,10.5000,-66.9167,America/Caracas,1739700900,2025-02-16 06:15:00,24.3,75.7,Clear,...,06:36 PM,09:53 PM,09:20 AM,Waning Gibbous,88,2025-02-16,2025-02-16,2025,2,16
53590,Vietnam,Hanoi,21.0333,105.8500,Asia/Bangkok,1739700900,2025-02-16 17:15:00,16.2,61.2,Light rain,...,05:55 PM,09:06 PM,08:29 AM,Waning Gibbous,91,2025-02-16,2025-02-16,2025,2,16
53591,Yemen,Sanaa,15.3547,44.2067,Asia/Aden,1739700900,2025-02-16 13:15:00,23.8,74.8,Sunny,...,06:07 PM,09:18 PM,08:43 AM,Waning Gibbous,90,2025-02-16,2025-02-16,2025,2,16
53592,Zambia,Lusaka,-15.4167,28.2833,Africa/Lusaka,1739700900,2025-02-16 12:15:00,22.9,73.3,Patchy rain nearby,...,06:38 PM,09:06 PM,08:59 AM,Waning Gibbous,90,2025-02-16,2025-02-16,2025,2,16


In [34]:
#Check whether there is null values

pandas_df.isnull().sum()

country                         0
location_name                   0
latitude                        0
longitude                       0
timezone                        0
last_updated_epoch              0
last_updated                    0
temperature_celsius             0
temperature_fahrenheit          0
condition_text                  0
wind_mph                        0
wind_kph                        0
wind_degree                     0
wind_direction                  0
pressure_mb                     0
pressure_in                     0
precip_mm                       0
precip_in                       0
humidity                        0
cloud                           0
feels_like_celsius              0
feels_like_fahrenheit           0
visibility_km                   0
visibility_miles                0
uv_index                        0
gust_mph                        0
gust_kph                        0
air_quality_Carbon_Monoxide     0
air_quality_Ozone               0
air_quality_Ni

In [35]:
#checking for duplicates
duplicates = pandas_df.duplicated()
pandas_df[duplicates]

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,sunset,moonrise,moonset,moon_phase,moon_illumination,date_only,date,year,month,day


In [36]:
#Calculate rolling averages and rolling sums
from pyspark.sql.window import Window
#Define window specifications
windowSpec = Window.orderBy("date").rowsBetween(-2, 0)  # Rolling window of 3 days
from pyspark.sql.functions import avg, sum, col
df_with_rolling_avg = df.withColumn("rolling_avg_temperature", avg("temperature_fahrenheit").over(windowSpec))
df_with_rolling_avg = df_with_rolling_avg.withColumn("rolling_avg_precip", avg("precip_in").over(windowSpec))
df_with_rolling_avg = df_with_rolling_avg.withColumn("rolling_sum_precip", sum("precip_in").over(windowSpec))
df_with_rolling_avg.show()

+-------------------+----------------+--------+---------+--------------------+------------------+-------------------+-------------------+----------------------+--------------------+--------+--------+-----------+--------------+-----------+-----------+---------+---------+--------+-----+------------------+---------------------+-------------+----------------+--------+--------+--------+---------------------------+-----------------+----------------------------+---------------------------+-----------------+----------------+------------------------+--------------------------+--------+--------+--------+----------+--------------+-----------------+----------+----------+----+-----+---+-----------------------+--------------------+--------------------+
|            country|   location_name|latitude|longitude|            timezone|last_updated_epoch|       last_updated|temperature_celsius|temperature_fahrenheit|      condition_text|wind_mph|wind_kph|wind_degree|wind_direction|pressure_mb|pressure_in|p